## Лабораторная работа №3

### Импорты

In [1]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from tensorflow.keras.models import Sequential, Model
from tensorflow.keras import layers
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.optimizers import RMSprop
from sklearn.utils import shuffle
from tensorflow.keras import optimizers
from tensorflow.keras.utils import plot_model

import numpy as np

from IPython.display import SVG

from datetime import datetime

### Загрузка данных

In [2]:
BATCH = 128
datagen = ImageDataGenerator(rescale=1./255)


    
train_data = datagen.flow_from_directory('/kaggle/input/intel-image-classification/seg_train/seg_train',
                                        target_size=(150, 150),
                                        batch_size=BATCH,
                                        class_mode='categorical',
                                        shuffle=True)

test_data = datagen.flow_from_directory('/kaggle/input/intel-image-classification/seg_test/seg_test',
                                        target_size=(150, 150),
                                        batch_size=BATCH,
                                        class_mode='categorical',
                                        shuffle=True)

Found 14034 images belonging to 6 classes.
Found 3000 images belonging to 6 classes.


In [3]:
train_data.class_indices

{'buildings': 0,
 'forest': 1,
 'glacier': 2,
 'mountain': 3,
 'sea': 4,
 'street': 5}

Зададим общее количество эпох для тестирования конфигураций равное **20**.

In [4]:
EPOCHS = 20

## GPU Info

In [5]:
import subprocess
import pprint

sp = subprocess.Popen(['nvidia-smi'], stdout=subprocess.PIPE, stderr=subprocess.PIPE)

out_str = sp.communicate()
out_list = str(out_str[0]).split('\\n')

out_dict = {}

for item in out_list:
    print(item)

b'Thu Nov 21 01:13:17 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.67       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  On   | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P0    25W / 250W |      0MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU

In [6]:
def saveModel(model, filename):
    model.summary()
    plot_model(model,to_file=filename,show_shapes=True, expand_nested=True)

### Конфигурация №1

In [7]:
model = Sequential([
    layers.Conv2D(32, (3, 3), activation = 'relu', input_shape = (150, 150, 3)),
    layers.MaxPooling2D(2, 2),
    layers.Conv2D(32, (3, 3), activation = 'relu'),
    layers.MaxPooling2D(2, 2),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(6, activation='softmax')
])
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics=['accuracy'])

saveModel(model, 'model1.png')

time_start = datetime.now()
history = model.fit_generator(train_data, steps_per_epoch=len(train_data), shuffle=True, epochs=EPOCHS, validation_steps=len(test_data), validation_data=test_data)
print('Time: ', datetime.now() - time_start)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 148, 148, 32)      896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 74, 74, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 72, 72, 32)        9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 36, 36, 32)        0         
_________________________________________________________________
flatten (Flatten)            (None, 41472)             0         
_________________________________________________________________
dense (Dense)                (None, 128)               5308544   
_________________________________________________________________
dense_1 (Dense)              (None, 6)                 7

### Конфигурация №2

In [8]:
model = Sequential([
    layers.Conv2D(128, (3, 3), activation = 'tanh', input_shape = (150, 150, 3)),
    layers.MaxPooling2D(2, 2),
    layers.Conv2D(64, (3, 3), activation = 'tanh'),
    layers.MaxPooling2D(2, 2),
    layers.Conv2D(32, (3, 3), activation = 'tanh'),
    layers.MaxPooling2D(2, 2),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(6, activation='softmax')
])
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics=['accuracy'])

saveModel(model, 'model2.png')

time_start = datetime.now()
history = model.fit_generator(train_data, steps_per_epoch=len(train_data), shuffle=True, epochs=EPOCHS, validation_steps=len(test_data), validation_data=test_data)
print('Time: ', datetime.now() - time_start)

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 148, 148, 128)     3584      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 74, 74, 128)       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 72, 72, 64)        73792     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 36, 36, 64)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 34, 34, 32)        18464     
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 17, 17, 32)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 9248)             

### Конфигурация №3

In [9]:
model = Sequential([
    layers.Conv2D(256, (3, 3), activation = 'relu', input_shape = (150, 150, 3)),
    layers.MaxPooling2D(5, 5),
    layers.Conv2D(128, (3, 3), activation = 'relu'),
    layers.MaxPooling2D(2, 2),
    layers.Conv2D(64, (3, 3), activation = 'relu'),
    layers.MaxPooling2D(2, 2),
    layers.Conv2D(32, (3, 3), activation = 'relu'),
    layers.MaxPooling2D(2, 2),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(6, activation='softmax')
])
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics=['accuracy'])

saveModel(model, 'model3.png')

time_start = datetime.now()
history = model.fit_generator(train_data, steps_per_epoch=len(train_data), shuffle=True, epochs=EPOCHS, validation_steps=len(test_data), validation_data=test_data)
print('Time: ', datetime.now() - time_start)

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 148, 148, 256)     7168      
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 29, 29, 256)       0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 27, 27, 128)       295040    
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 13, 13, 128)       0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 11, 11, 64)        73792     
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 3, 3, 32)         

### Конфигурация №4

In [10]:
model = Sequential([
    layers.Conv2D(256, (3, 3), activation = 'relu', input_shape = (150, 150, 3)),
    layers.MaxPooling2D(5, 5),
    layers.Conv2D(180, (3, 3), activation = 'relu'),
    layers.MaxPooling2D(2, 2),
    layers.Conv2D(128, (3, 3), activation = 'relu'),
    layers.Conv2D(64, (3, 3), activation = 'relu'),
    layers.Conv2D(32, (3, 3), activation = 'relu'),
    layers.MaxPooling2D(2, 2),
    layers.Flatten(),
    layers.Dense(180,activation='relu'),
    layers.Dense(128, activation='relu'),
    layers.Dense(6, activation='softmax')
])
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics=['accuracy'])

saveModel(model, 'model4.png')

time_start = datetime.now()
history = model.fit_generator(train_data, steps_per_epoch=len(train_data), shuffle=True, epochs=EPOCHS, validation_steps=len(test_data), validation_data=test_data)
print('Time: ', datetime.now() - time_start)

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_9 (Conv2D)            (None, 148, 148, 256)     7168      
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 29, 29, 256)       0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 27, 27, 180)       414900    
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 13, 13, 180)       0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 11, 11, 128)       207488    
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 9, 9, 64)          73792     
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 7, 7, 32)         

### Конфигурация №5

In [11]:
model = Sequential([
    layers.Conv2D(256, (3, 3), activation = 'relu', input_shape = (150, 150, 3)),
    layers.Conv2D(180, (3, 3), activation = 'relu'),
    layers.MaxPooling2D(5, 5),
    layers.Conv2D(180, (3, 3), activation = 'relu'),
    layers.Conv2D(128, (3, 3), activation = 'relu'),
    layers.MaxPooling2D(2, 2),
    layers.Conv2D(64, (3, 3), activation = 'relu'),
    layers.MaxPooling2D(2, 2),
    layers.Conv2D(32, (3, 3), activation = 'relu'),
    layers.MaxPooling2D(2, 2),
    layers.Flatten(),
    layers.Dense(180,activation='relu'),
    layers.Dense(128, activation='relu'),
    layers.Dense(50,activation='relu'),
    layers.Dropout(rate=0.5),
    layers.Dense(6, activation='softmax')
])
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics=['accuracy'])

saveModel(model, 'model5.png')

time_start = datetime.now()
history = model.fit_generator(train_data, steps_per_epoch=len(train_data), shuffle=True, epochs=EPOCHS, validation_steps=len(test_data), validation_data=test_data)
print('Time: ', datetime.now() - time_start)

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_14 (Conv2D)           (None, 148, 148, 256)     7168      
_________________________________________________________________
conv2d_15 (Conv2D)           (None, 146, 146, 180)     414900    
_________________________________________________________________
max_pooling2d_12 (MaxPooling (None, 29, 29, 180)       0         
_________________________________________________________________
conv2d_16 (Conv2D)           (None, 27, 27, 180)       291780    
_________________________________________________________________
conv2d_17 (Conv2D)           (None, 25, 25, 128)       207488    
_________________________________________________________________
max_pooling2d_13 (MaxPooling (None, 12, 12, 128)       0         
_________________________________________________________________
conv2d_18 (Conv2D)           (None, 10, 10, 64)       

### Конфигурация №6 - tanh everywhere

In [12]:
model = Sequential([
    layers.Conv2D(256, (3, 3), activation = 'tanh', input_shape = (150, 150, 3)),
    layers.Conv2D(180, (3, 3), activation = 'tanh'),
    layers.MaxPooling2D(5, 5),
    layers.Conv2D(180, (3, 3), activation = 'tanh'),
    layers.Conv2D(128, (3, 3), activation = 'tanh'),
    layers.MaxPooling2D(2, 2),
    layers.Conv2D(64, (3, 3), activation = 'tanh'),
    layers.MaxPooling2D(2, 2),
    layers.Conv2D(32, (3, 3), activation = 'tanh'),
    layers.MaxPooling2D(2, 2),
    layers.Flatten(),
    layers.Dense(180,activation='tanh'),
    layers.Dense(128, activation='tanh'),
    layers.Dense(50,activation='tanh'),
    layers.Dropout(rate=0.5),
    layers.Dense(6, activation='softmax')
])
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics=['accuracy'])

saveModel(model, 'model6.png')

time_start = datetime.now()
history = model.fit_generator(train_data, steps_per_epoch=len(train_data), shuffle=True, epochs=EPOCHS, validation_steps=len(test_data), validation_data=test_data)
print('Time: ', datetime.now() - time_start)

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_20 (Conv2D)           (None, 148, 148, 256)     7168      
_________________________________________________________________
conv2d_21 (Conv2D)           (None, 146, 146, 180)     414900    
_________________________________________________________________
max_pooling2d_16 (MaxPooling (None, 29, 29, 180)       0         
_________________________________________________________________
conv2d_22 (Conv2D)           (None, 27, 27, 180)       291780    
_________________________________________________________________
conv2d_23 (Conv2D)           (None, 25, 25, 128)       207488    
_________________________________________________________________
max_pooling2d_17 (MaxPooling (None, 12, 12, 128)       0         
_________________________________________________________________
conv2d_24 (Conv2D)           (None, 10, 10, 64)       

### Конфигурация №7 - average pooling

In [13]:
model = Sequential([
    layers.Conv2D(256, (3, 3), activation = 'tanh', input_shape = (150, 150, 3)),
    layers.Conv2D(180, (3, 3), activation = 'tanh'),
    layers.AveragePooling2D(5, 5),
    layers.Conv2D(180, (3, 3), activation = 'tanh'),
    layers.Conv2D(128, (3, 3), activation = 'tanh'),
    layers.AveragePooling2D(2, 2),
    layers.Conv2D(64, (3, 3), activation = 'tanh'),
    layers.AveragePooling2D(2, 2),
    layers.Conv2D(32, (3, 3), activation = 'tanh'),
    layers.AveragePooling2D(2, 2),
    layers.Flatten(),
    layers.Dense(180,activation='tanh'),
    layers.Dense(128, activation='tanh'),
    layers.Dense(50,activation='tanh'),
    layers.Dropout(rate=0.5),
    layers.Dense(6, activation='softmax')
])
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics=['accuracy'])

saveModel(model, 'model7.png')

time_start = datetime.now()
history = model.fit_generator(train_data, steps_per_epoch=len(train_data), shuffle=True, epochs=EPOCHS, validation_steps=len(test_data), validation_data=test_data)
print('Time: ', datetime.now() - time_start)

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_26 (Conv2D)           (None, 148, 148, 256)     7168      
_________________________________________________________________
conv2d_27 (Conv2D)           (None, 146, 146, 180)     414900    
_________________________________________________________________
average_pooling2d (AveragePo (None, 29, 29, 180)       0         
_________________________________________________________________
conv2d_28 (Conv2D)           (None, 27, 27, 180)       291780    
_________________________________________________________________
conv2d_29 (Conv2D)           (None, 25, 25, 128)       207488    
_________________________________________________________________
average_pooling2d_1 (Average (None, 12, 12, 128)       0         
_________________________________________________________________
conv2d_30 (Conv2D)           (None, 10, 10, 64)       